In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Reading the dataset

In [2]:
df=pd.read_csv(r"C:\classes\fake_news_detection.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

# Checking a random article

In [5]:
df.shape

(20800, 5)

In [6]:
from random import randint
random=randint(0,20800)
df["title"][random]

'Trump Promises ‘New Deal for Black America’'

In [7]:
df["label"][random]

1

# Filtering the words

In [8]:
import re
ps=PorterStemmer()
tokens=[]
for i in range(0,len(df)):
    token=re.sub('[^a-zA-Z]',' ',str(df["title"][i]))
    token=token.lower()
    token=token.split()
    token=[ps.stem(word) for word in token if not word in stopwords.words('english')]
    token=' '.join(token)
    tokens.append(token)

In [9]:
tokens[random]

'trump promis new deal black america'

## Count Vectorization

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

x=cv.fit_transform(tokens)

In [11]:
y=df["label"]

# Train test split

In [12]:
from sklearn.model_selection import train_test_split as tts
train_x,test_x,train_y,test_y=tts(x,y,test_size=0.1,random_state=20)

# Model Building

## Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

lr.fit(train_x,train_y)

pred_lr=lr.predict(test_x)
pred_lr=pd.DataFrame(pred_lr,columns=["pred"])

lr.score(test_x,test_y)

0.9403846153846154

## Passive Aggressive Classifier

In [14]:
from sklearn.linear_model import PassiveAggressiveClassifier
ps=PassiveAggressiveClassifier()

ps.fit(train_x,train_y)

pred_ps=ps.predict(test_x)
pred_ps=pd.DataFrame(pred_ps,columns=["pred"])

ps.score(test_x,test_y)

0.9298076923076923

## RandomForest Classifier

In [15]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()

rf.fit(train_x,train_y)

pred_rf=rf.predict(test_x)
pred_rf=pd.DataFrame(pred_rf,columns=["pred"])

rf.score(test_x,test_y)

0.9394230769230769

## Ensembling the models

In [16]:
from statistics import mode
ensembled_pred=[]
for i in range(0,len(test_y)):
    ensembled_pred.append(mode([pred_lr["pred"][i],pred_rf["pred"][i],pred_ps["pred"][i]]))

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(ensembled_pred,test_y)

0.9427884615384615